### Better way 32. 긴 리스트 컴프리헨션보다는 제너레이터 식을 사용하라

- 파일의 각 라인마다 읽어 글자 수를 반환 하는 컴프리헨션을 작성한다고 생각 해보자, 
만약 파일 라인이 수 없이 많고 다 읽기 어려운 분량의 파일이라면 컴프리헨션을 사용하는 데 문제가 있다. 
    - 그래서 입력 양이 적은 경우는 컴프리헨션을 사용 할 수 있지만, 방대한 양이라면 제너레이터 식을 이용하자.

> List Comprehension

In [3]:
value = [len(x) for x in open("my_file.txt")]

In [4]:
value

[100, 57, 15, 1, 12, 75, 5, 86, 89, 10]

> Generator식 
- 제너레이터식을 사용하면, 시퀀스 값 전체가 나타나지 않고 원소를 하나씩 볼 수 있는 이터레이터가 생성 된다. 
    - 위 제너레이터를 출력하려면 next()를 이용하여, 전체를 호출 하지 않고 하나씩 출력 할 수 있다.
<p></p>
- 제너레이터의 강점은 제너레이터 식이 반환 한 이터레이터를 다른 제너레이터와 상호 작용을 할 수 있다.

In [51]:
gen_value = (len(x) for x in open("my_file.txt"))

In [16]:
# next(gen_value)
roots = ({x : x**0.5}for x in gen_value)
print(next(roots))

{86: 9.273618495495704}


### Better way 33. yield from을 사용 해 여러 제너레이터를 합성하라

In [38]:
def move(period, speed) :
    for _ in range(period) :
        yield speed

def pause(delay) :
    for _ in range(delay) :
        yield 0

def animate() :
    for delta in move(4, 5.0) :
        yield delta
    for delta in pause(3) :
        yield delta
    for delta in move(2, 3.0) :
        yield delta

def render(delta) :
    print(f"delta : {delta:.1f}")

def run(func) :
    for delta in func() :
        render(delta)

In [39]:
run(animate)

delta : 5.0
delta : 5.0
delta : 5.0
delta : 5.0
delta : 0.0
delta : 0.0
delta : 0.0
delta : 3.0
delta : 3.0


In [40]:
def animate_composed() :
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 3.0)
    

In [41]:
run(animate_composed)

delta : 5.0
delta : 5.0
delta : 5.0
delta : 5.0
delta : 0.0
delta : 0.0
delta : 0.0
delta : 3.0
delta : 3.0


In [ ]:
def read_line() :
    yield from gen_value

print_line = read_line()
next(print_line)

In [64]:
import timeit

def child() :
    for i in range(1_000_000) :
        yield i

def slow() :
    for i in child() :
        yield i

def fast() :
    yield from child()

baseline = timeit.timeit(
    stmt = "for _ in slow(): pass",
    globals = globals(),
    number = 50)

print(f"수동 내포 : {baseline:.2f}s")        
comparison = timeit.timeit(
    stmt = "for _ in fast() : pass",
    globals = globals(),
    number = 50
)
print(f"합성 사용 : {comparison:.2f}s")

reduction = -(comparison - baseline) / baseline
print(f"{reduction:.1%} 시간이 적게 듦")

수동 내포 : 5.88s
합성 사용 : 4.89s
16.9% 시간이 적게 듦
